In [63]:
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import argparse

In [2]:
#not needed
def customOneHotEncoder(data):
    dataAdjust = data.ljust(200,'0')[:200] # padding if not of length and adjusting the data lenght to get a 200x39 input matrix
    # define universe of possible input values
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz,._-/+'
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in dataAdjust]
    #print(integer_encoded)
    # one hot encode
    onehot_encoded = list()
    for i, value in enumerate(integer_encoded):
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    #print(onehot_encoded) # the real encoding
    return onehot_encoded

# takes a .csv filename
def dataPreprocessing(fileName):
    df = pd.read_csv(fileName, header = None)
    
    #prepare the imput data
    xString = df.iloc[:,:43].to_string(header=False, index=False, index_names = False).split('\n')
    xList = [','.join(ele.split()) for ele in xString] # gives comma separated strings for each row of DataFrame
    xData = []
    for string in xList:
        stringLower = string.lower()
        oneHot = customOneHotEncoder(stringLower)
        xData.append(oneHot)
    xMid = np.array(xData)
    xArray = xMid.transpose(0,2,1) # convert xMid's dim (size, 200, 39) to (size, 39, 200)
    
    #prepare the label data
    df[43] = np.where(pd.isnull(df[47]), 'Normal', 'attack') # replacing anything except 'normal' with 'attack'
    Ydf = df[43]
    #labelName = Ydf.unique().tolist().sort() # sorted 38 label names
    #yArray = Ydf.str.get_dummies().to_numpy() # ndarray of shape(rows/lines, 38)
    yArray = Ydf.to_numpy()
    
    assert xArray.shape[0] == yArray.shape[0], 'unequal input and label sample size'
    
    
    return xArray, yArray # return processed array of input and label

In [86]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

class UNSWDataset(Dataset):
    def __init__(self, fileName, seq_len):
        self.data = pd.read_csv(fileName, header = None)
        self.seq_len = seq_len
        #word embedding here
        idx= [0,1,9]
        for i in idx:
            self.data[i] = self.data[i].astype('category')
            self.data[i] = self.data[i].cat.codes
        self.num_seq = len(self.data)-self.seq_len
        self.xArray = np.zeros((self.num_seq, 41, self.seq_len))
        self.yArray = np.zeros(self.num_seq)
        for i in range(len(self.data)):
            if i >= self.seq_len:
                self.xArray[i-self.seq_len] = self.data.iloc[i-self.seq_len:i,:-1].to_numpy().transpose()
                #self.data replaces xarray
                self.yArray[i-self.seq_len] = self.data.iloc[i,-1]
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # prepare x data
        #string = list(','.join('%s' %x for x in y) for y in self.data.iloc[[idx], :47].values)
        #stringLower = string[0].lower()
        #xData = customOneHotEncoder(stringLower) # Dim (200, 39)
        #xMid = np.array(xData)
        #xArray = xMid.transpose() # should be now (39, 200)
        
        #xArray = self.data.iloc[idx:self.seq_len + idx,:-1].to_numpy() 
        
        # prepate y data
        #self.data.iloc[idx, 41] = np.where(self.data.iloc[idx, 41]=='normal', 0, 1) # replacing normals with 0 and anything else with 1
        #yArray = np.where(pd.isnull(self.data.iloc[idx, 47]), 0, 1)
        #yArray = self.data.iloc[idx+self.seq_len,-1]
        #yArray = Ydf.to_numpy()
        a = self.xArray[idx]
        b = self.yArray[idx]
        #assert xArray.shape == yArray.shape, 'unequal input and label sample size'
        #if(idx>=self.seq_len):
         #   xArray = self.data.iloc[idx-self.seq_len:idx,:-1].to_numpy() 
          #  yArray = self.data.iloc[idx,-1]
        
        return torch.from_numpy(self.xArray[idx]), self.yArray[idx] # returns torch tensor of x and y

In [88]:
params = {'batch_size': 64}
seq_len = 270
fileNameTrain = 'UNSW-NB15_1_reduced-16-binary-class-redcol.csv'
fileNameTest = 'UNSW-NB15_1_reduced-16-binary-class-redcol.csv'
datasetTrain = UNSWDataset(fileNameTrain, seq_len)
datasetTest = UNSWDataset(fileNameTest, seq_len)
dataGeneratorTrain = DataLoader(datasetTrain, **params)

#define sequence



In [70]:
for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
    #print (batch_idx, data, target)
    print(data.shape)
    print(target.shape)

0 tensor([[1.1000e+01, 1.1000e+01, 1.1000e+01,  ..., 1.0000e+01, 1.0000e+01,
         1.0000e+01],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 3.0000e+00,
         3.0000e+00],
        [1.0550e-03, 3.6133e-02, 1.1190e-03,  ..., 1.0367e+01, 1.0930e+01,
         3.7953e-02],
        ...,
        [1.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 3.0000e+00, 4.0000e+00,
         3.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 2.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         4.0000e+00]], dtype=torch.float64) tensor(0., dtype=torch.float64)
1 tensor([[1.1000e+01, 1.1000e+01, 1.1000e+01,  ..., 1.0000e+01, 1.0000e+01,
         1.0000e+01],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 3.0000e+00,
         3.0000e+00],
        [3.6133e-02, 1.1190e-03, 1.2090e-03,  ..., 1.0930e+01, 3.7953e-02,
         2.7114e-02],
        ...,
        [1.0000e+00, 2.0000e+00, 1.0000e+00

44 tensor([[1.1000e+01, 1.1000e+01, 1.1000e+01,  ..., 1.0000e+01, 1.0000e+01,
         1.0000e+01],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 3.0000e+00,
         3.0000e+00],
        [1.1520e-03, 4.9960e-03, 1.0440e-03,  ..., 2.0127e+00, 9.1131e+00,
         1.1385e+01],
        ...,
        [1.0000e+00, 1.0000e+00, 2.0000e+00,  ..., 3.0000e+00, 6.0000e+00,
         4.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [1.0000e+00, 2.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         4.0000e+00]], dtype=torch.float64) tensor(0., dtype=torch.float64)
45 tensor([[1.1000e+01, 1.1000e+01, 1.1000e+01,  ..., 1.0000e+01, 1.0000e+01,
         1.0000e+01],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 3.0000e+00,
         3.0000e+00],
        [4.9960e-03, 1.0440e-03, 1.0890e-03,  ..., 9.1131e+00, 1.1385e+01,
         1.3468e+01],
        ...,
        [1.0000e+00, 2.0000e+00, 1.0000e+

         1.0000e+00]], dtype=torch.float64) tensor(0., dtype=torch.float64)
107 tensor([[1.1000e+01, 1.1000e+01, 1.0000e+01,  ..., 1.0000e+01, 1.0000e+01,
         1.0000e+01],
        [1.0000e+00, 1.0000e+00, 3.0000e+00,  ..., 3.0000e+00, 3.0000e+00,
         3.0000e+00],
        [1.0670e-03, 1.1850e-03, 2.4900e-01,  ..., 1.8098e+00, 9.7131e+00,
         4.2580e-01],
        ...,
        [3.0000e+00, 3.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 2.0000e+00,
         7.0000e+00],
        [1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         1.0000e+00],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 1.0000e+00, 1.0000e+00,
         4.0000e+00]], dtype=torch.float64) tensor(0., dtype=torch.float64)
108 tensor([[1.1000e+01, 1.0000e+01, 1.1000e+01,  ..., 1.0000e+01, 1.0000e+01,
         1.0000e+01],
        [1.0000e+00, 3.0000e+00, 4.0000e+00,  ..., 3.0000e+00, 3.0000e+00,
         3.0000e+00],
        [1.1850e-03, 2.4900e-01, 1.7000e-05,  ..., 9.7131e+00, 4.2580e-01

torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size(

torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size(

torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size([64, 41, 270])
torch.Size([64])
torch.Size(

IndexError: index 40492 is out of bounds for axis 0 with size 40492

In [4]:
params = {'batch_size': 64, 'shuffle': True}
filedataset = 'UNSW-NB15_1.csv'
dataset = NSLKDDDataset(filedataset)
total_length = dataset.__len__()
train_length = int(0.6 * total_length)
test_length = int(0.2 * total_length)
val_length = total_length - train_length - test_length
datasetTrain, datasetVal, datasetTest = torch.utils.data.random_split(dataset, [train_length, val_length, test_length])
dataGeneratorTrain = DataLoader(datasetTrain, **params)
dataGeneratorTest = DataLoader(datasetTest, **params)
dataGeneratorVal = DataLoader(datasetVal, **params)

C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (1,3,47) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
datasetTrain.__len__()

420000

In [75]:
root = './data/mnist'
batch_size = 64
n_classes = 2
input_channels = 41
seq_length = 270
epochs = 10
steps = 0

In [76]:
#train_loader, test_loader = data_generator(root, batch_size)

permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [32] * 6 #hidden nodes times levels 
kernel_size = 5
nheads = 1

model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=0.25)
#input_size, output_size, num_channels, 
#kernel_size, dropout, num_sub_blocks, temp_attn, nheads, en_res,                              num_sub_blocks
#conv, key_size
#
# nheads, key_size


lr = 1e-5
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)

In [82]:
def train(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        optimizer.zero_grad()
        #print(data.shape)
        data = data.view(-1, input_channels, seq_length)
        data, target = Variable(data), Variable(target)
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        #print(target)
        optimizer.zero_grad()
        #print(data[0])
        data = data.type(torch.FloatTensor)
        output = model(data)
        #print(output.shape)
        target = target.type(torch.LongTensor)
        loss1 = torch.nn.CrossEntropyLoss()
        loss = F.nll_loss(output, target) # negative log likelihood
        loss = loss1(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss
        steps += seq_length
        #print(batch_idx)
        if batch_idx > 0 and batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(dataGeneratorTrain.dataset),
                100. * batch_idx / len(dataGeneratorTrain), train_loss.item()/100, steps))
            train_loss = 0

In [78]:
def test():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorTest:
            model.eval()
            #print('checkpoint 1')
            #data = data.view(-1, input_channels, seq_length)
            #print('checkpoint 2')
            data = data.type(torch.FloatTensor)
            #print('checkpoint 3')
            target = target.type(torch.LongTensor)
            #print('checkpoint 4')
            data, target = Variable(data, volatile=True), Variable(target)
            #print('checkpoint 5')
            output = model(data)
            #print('checkpoint 6')
            #loss1 = torch.nn.CrossEntropyLoss()
            test_loss += F.nll_loss(output, target, size_average=False).item()
            #print('checkpoint 7')
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            #print('checkpoint 8')
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            #print('checkpoint 9')

        test_loss /= len(dataGeneratorTest.dataset)
        #print('checkpoint 10')
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorTest.dataset),
            100. * correct / len(dataGeneratorTest.dataset)))
        return test_loss

In [83]:
if __name__ == "__main__":
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Train Epoch: 1 [3200/40762 (8%)]	Loss: 1053508.240000	Steps: 112590
Train Epoch: 1 [6400/40762 (16%)]	Loss: 1996061.760000	Steps: 126090
Train Epoch: 1 [9600/40762 (24%)]	Loss: 2660583.680000	Steps: 139590
Train Epoch: 1 [12800/40762 (31%)]	Loss: 2587426.080000	Steps: 153090
Train Epoch: 1 [16000/40762 (39%)]	Loss: 2152780.320000	Steps: 166590
Train Epoch: 1 [19200/40762 (47%)]	Loss: 1933876.480000	Steps: 180090
Train Epoch: 1 [22400/40762 (55%)]	Loss: 1264247.360000	Steps: 193590
Train Epoch: 1 [25600/40762 (63%)]	Loss: 1602323.840000	Steps: 207090
Train Epoch: 1 [28800/40762 (71%)]	Loss: 2071114.400000	Steps: 220590
Train Epoch: 1 [32000/40762 (78%)]	Loss: 1421504.320000	Steps: 234090
Train Epoch: 1 [35200/40762 (86%)]	Loss: 1569503.200000	Steps: 247590
Train Epoch: 1 [38400/40762 (94%)]	Loss: 1440727.360000	Steps: 261090


IndexError: index 40492 is out of bounds for axis 0 with size 40492

In [12]:
#NEED TO DO THIS ONE
from sklearn.metrics import classification_report
def get_metrics(dataLoader):
    total = 0
    correct = 0
    precision = 0
    recall = 0
    f1_score = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in dataLoader: # just 1 batch
            model.eval()
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            #print(predicted)
            total+=target.size(0)
            correct+=(predicted == target).sum().item()
            report = classification_report(target, predicted, output_dict=True)
            precision += report['macro avg']['precision']
            recall += report['macro avg']['recall']
            f1_score += report['macro avg']['f1-score']
            accuracy += report['accuracy']
            print(report)
    print("Precision: {}, Recall: {}, F1-Score: {}, Accuracy: {}, AccuracyCust: {}".format(precision, recall, f1_score, accuracy, correct/total))
    

In [ ]:
#for now don't do thisone
#fileNameVal = 'UNSW_NB15_testing-set.csv'
#datasetVal = NSLKDDDataset(fileNameVal)
params = {'batch_size': 22544, 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
#RuntimeError: expected scalar type Int but found Float
#get_metrics(dataGeneratorVal)
def val():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorVal:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            #test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            print(pred)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorVal.dataset),
            100. * correct / len(dataGeneratorVal.dataset)))
        return test_loss

val()


In [13]:
#do this one last
params = {'batch_size': datasetVal.__len__(), 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
get_metrics(dataGeneratorVal)

{'0': {'precision': 0.999888706697631, 'recall': 0.9937102280687524, 'f1-score': 0.9967898933416175, 'support': 135617}, '1': {'precision': 0.8366526235158943, 'recall': 0.9965784671532847, 'f1-score': 0.9096398084530501, 'support': 4384}, 'accuracy': 0.9938000442853979, 'macro avg': {'precision': 0.9182706651067627, 'recall': 0.9951443476110186, 'f1-score': 0.9532148508973338, 'support': 140001}, 'weighted avg': {'precision': 0.9947771218613174, 'recall': 0.9938000442853979, 'f1-score': 0.9940608701764154, 'support': 140001}}
Precision: 0.9182706651067627, Recall: 0.9951443476110186, F1-Score: 0.9532148508973338, Accuracy: 0.9938000442853979, AccuracyCust: 0.9938000442853979


In [14]:
#do this one
params = {'batch_size': datasetTest.__len__(), 'shuffle': True}
dataGeneratorTest = DataLoader(datasetTest, **params)
get_metrics(dataGeneratorTest)

{'0': {'precision': 0.9998960607005509, 'recall': 0.9939043289596032, 'f1-score': 0.9968911917098446, 'support': 135506}, '1': {'precision': 0.8443271767810027, 'recall': 0.9968847352024922, 'f1-score': 0.9142857142857144, 'support': 4494}, 'accuracy': 0.994, 'macro avg': {'precision': 0.9221116187407767, 'recall': 0.9953945320810478, 'f1-score': 0.9555884529977795, 'support': 140000}, 'weighted avg': {'precision': 0.9949022995267334, 'recall': 0.994, 'f1-score': 0.99423955588453, 'support': 140000}}
Precision: 0.9221116187407767, Recall: 0.9953945320810478, F1-Score: 0.9555884529977795, Accuracy: 0.994, AccuracyCust: 0.994


In [17]:
torch.save(model, "tcn_unsw.pth")

In [90]:
import pandas as pd

data = pd.read_csv("UNSW-NB15_1_reduced-16-binary-class-redcol.csv", header=None)
data.describe(include='all')

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
count,40762,40762,40762.000000,4.076200e+04,4.076200e+04,40762.000000,40762.000000,40762.000000,40762.000000,40762,...,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000
unique,13,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,tcp,FIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,29688,29153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.114260,5.954914e+03,4.047243e+04,54.699818,47.195918,6.153157,18.010402,NaN,...,0.016044,0.017001,5.043717,4.739095,3.542245,4.039399,1.606619,1.046906,1.909327,0.079314
std,NaN,NaN,5.039567,8.336349e+04,1.862701e+05,68.360785,62.596066,32.761024,66.327121,NaN,...,0.125648,0.148529,4.012933,3.738378,2.614704,2.996480,1.672633,0.307345,1.489764,0.270232
min,NaN,NaN,0.000000,4.600000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,NaN,NaN,0.004464,5.200000e+02,3.040000e+02,31.000000,29.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,0.000000
50%,NaN,NaN,0.036865,1.540000e+03,1.956000e+03,31.000000,29.000000,4.000000,4.000000,NaN,...,0.000000,0.000000,4.000000,4.000000,3.000000,3.000000,1.000000,1.000000,1.000000,0.000000
75%,NaN,NaN,0.528106,3.806000e+03,1.016800e+04,31.000000,29.000000,7.000000,11.000000,NaN,...,0.000000,0.000000,7.000000,7.000000,5.000000,5.000000,1.000000,1.000000,2.000000,0.000000


In [20]:
for i in range(41):
    print(data[i].isnull().values.any())

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40762 entries, 0 to 40761
Data columns (total 42 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       40762 non-null  object 
 1   1       40762 non-null  object 
 2   2       40762 non-null  float64
 3   3       40762 non-null  int64  
 4   4       40762 non-null  int64  
 5   5       40762 non-null  int64  
 6   6       40762 non-null  int64  
 7   7       40762 non-null  int64  
 8   8       40762 non-null  int64  
 9   9       40762 non-null  object 
 10  10      40762 non-null  float64
 11  11      40762 non-null  float64
 12  12      40762 non-null  int64  
 13  13      40762 non-null  int64  
 14  14      40762 non-null  int64  
 15  15      40762 non-null  int64  
 16  16      40762 non-null  int64  
 17  17      40762 non-null  int64  
 18  18      40762 non-null  int64  
 19  19      40762 non-null  int64  
 20  20      40762 non-null  int64  
 21  21      40762 non-null  int64  
 22

In [27]:
data[0] = data[0].astype('category')
data[0].cat.codes


0        11
1        11
2        11
3        11
4        11
         ..
40757    10
40758    10
40759    10
40760    10
40761    11
Length: 40762, dtype: int8

In [28]:
data[0].unique().shape

(13,)

In [29]:
data[1] = data[1].astype('category')
data[1].cat.codes
data[1].unique().shape

(9,)

In [30]:
data[9] = data[9].astype('category')
data[9].cat.codes
data[9].unique().shape

(13,)

In [36]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,11,1,0.001055,132,164,31,29,0,0,2,...,0,0,3,7,1,3,1,1,1,0
1,11,1,0.036133,528,304,31,29,0,0,0,...,0,0,2,4,2,3,1,1,2,0
2,11,1,0.001119,146,178,31,29,0,0,2,...,0,0,12,8,1,2,2,1,1,0
3,11,1,0.001209,132,164,31,29,0,0,2,...,0,0,6,9,1,1,1,1,1,0
4,11,1,0.001169,146,178,31,29,0,0,2,...,0,0,7,9,1,1,1,1,1,0


In [35]:
idx= [0,1,9]
for i in idx:
    data[i] = data[i].astype('category')
    data[i] = data[i].cat.codes